# Road Following + Waypoint Detection on Jetbot

In this notebook we will be combining both optimized regression and classification models into one notebook so that our Jetbot will be able to perform *road following* as well as enable *waypoint detection* at the same time so that it can avoid collisions with obstacles that come on its way in real-time.

To begin, let's execute the code below to transfer the device from CPU memory to the GPU device.

In [ ]:
import torch
device = torch.device('cuda')

### **Road Following**   
- Upload the "*best_steering_model_xy.pth*" model file obtained from the `road_train_model.ipynb` into this notebooks's directory.

### **Waypoint Detection**             
- Upload the "*best_waypoint_model.pth*" model file obtained from the `waypoint_train_model.ipynb` into this notebooks's directory.

Load the TRT optimized models by executing the cell below

In [ ]:
import torch
import torchvision

model_road = torchvision.models.resnet18(pretrained=False)
model_road.fc = torch.nn.Linear(512, 2)
model_road.load_state_dict(torch.load('best_steering_model_xy.pth'))

model_waypoint = torchvision.models.alexnet(pretrained=False)
model_waypoint.classifier[6] = torch.nn.Linear(model_waypoint.classifier[6].in_features, 2)
model_waypoint.load_state_dict(torch.load('best_waypoint_model.pth'))

### Creating the Pre-Processing Function

We have now loaded our models, but there's a slight issue. The format that we trained our models on doesn't exactly match the format of the camera. To do that, we need to do some preprocessing. This involves the following steps:

1. Convert from HWC layout to CHW layout
2. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
3. Transfer the data from CPU memory to GPU memory
4. Add a batch dimension

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Awesome! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera.

In [ ]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224, fps=10)
image_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

We'll also create our robot instance which we'll need to drive the motors.

In [ ]:
from jetbot import Robot

robot = Robot()

Initialising NurseryHome environment.

In [ ]:
from environment import initialise, NurseryHome, PDDLReader

init = initialise.Initialise()
init.setup()

home = init.home

Now, we will define some sliders to control the JetBot
> **Note**: We have initialized the slider values for best known configurations, however these might not work for your dataset, therefore please increase or decrease the sliders according to your setup and environment

1. Speed Control slider: To start your JetBot increase ``speed_control_slider`` 
2. Steering Gain slider: If you see your JetBot is woblling, you need to reduce ``steering_gain_slider`` till it is smooth
3. Steering Bias slider: If you see your JetBot is biased towards extreme right or extreme left side of the track, you should control this slider till JetBot start following line or track in the center.  This accounts for motor biases as well as camera offsets

> Note: You should play around above mentioned sliders with lower speed to get smooth JetBot road following behavior.

4. Waypoint slider: Display the probability in which there is a waypoint in the front of the Jetbot using the waypoint detection model
5. Time for stop slider: To manually set the time for which the jetbot should remain stopped after an object has been removed
6. Waypoint threshold slider: To manually set the waypoint detection threshold to stop the Jetbot after a waypoint has been detected 

In [ ]:
# Road Following sliders
speed_control_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed control')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.04, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_control_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

# Waypoint sliders & buttons
waypoint_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, orientation='horizontal', description='waypoint')
button_layout = ipywidgets.Layout(width='128px', height='64px')
resume_button = ipywidgets.Button(description='resume', button_style='success', layout=button_layout)

stop = False
def on_resume_clicked():
    global stop
    stop = False
resume_button.on_click(on_resume_clicked)

display(ipywidgets.HBox([waypoint_slider, resume_button]))

# room number & name
room_slider = ipywidgets.IntSlider(min=0, max=home.numRoom()-1, orientation='horizontal', description='waypoint')
room_name_widget = ipywidgets.Text(description='room name')

curr_room = home.findRoomFromIndex(room_slider.value)
def on_room_slider_change(change):
    new_index = change['new']
    room_name_widget.value = home.findRoomFromIndex(new_index).name
    curr_room = home.findRoomFromIndex(new_index)

room_name_widget.value = home.findRoomFromIndex(room_slider.value).name
room_slider.observe(on_room_slider_change, names='value')

display(ipywidgets.HBox([room_slider, room_name_widget]))

Initalising robot and PDDL reader.

In [ ]:
from environment import Robot as RobotEntity

robot_entity = RobotEntity.Robot(home, curr_room)
pddlReader = PDDLReader.PDDLReader(robot_entity)

Next, we'll create a function that will get called whenever the camera's value changes. This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network models for Road following and Waypoint Detection
3. Check an if statements which would allow the Jetbot to perform road following and stop whenever a waypoint has been detected 
4. Compute the approximate steering value
5. Control the motors using proportional / derivative control (PD)

In [ ]:
angle = 0.0
angle_last = 0.0

clocwise = True

def execute(change):
    global angle, angle_last, speed_value
    global blocked_slider, robot, stop
    global clocwise

    image = change['new']
    image = preprocess(image)
    
    # WAYPOINT MODEL
    model_waypoint.cuda()
    y_w = model_waypoint(image.float())
    y_w = F.softmax(y_w, dim=1)
    prob_waypoint = float(y_w.flatten()[0])
    waypoint_slider.value = prob_waypoint
    
    # waypoint reached
    if prob_waypoint > 0.5:
        stop = True
        print("WAYPOINT REACHED")
        if clocwise:
            room_slider.value += 1
        else:
            room_slider.value -= 1
    
    # stop
    if stop:
        speed_slider.value = 0
    else:
        speed_slider.value = speed_control_slider.value
    
    # ROAD FOLLOWING MODEL
    xy = model_road(image).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    
execute({'new': camera.value})

Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing.

We accomplish that with the observe function.

>WARNING: This code will move the robot!! Please make sure your robot has clearance and it is on Lego or Track you have collected data on. The road follower and collision avoider should work, but the neural network is only as good as the data it's trained on!

In [ ]:
camera.observe(execute, names='value')

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame. 

You can now place JetBot on  Lego or Track you have collected data on and see whether it can follow the track and avoid collisions effectively.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [ ]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()